In [ ]:
import time
import torch
import sys
import os
import matplotlib.pyplot as plt

from ase import Atoms
from batch_nl import NeighbourList

from vesin import ase_neighbor_list as vesin_ase_neighbor_list
from matscipy.neighbours import neighbour_list as matscipy_neighbour_list
from torch_nl import compute_neighborlist, ase2data
from nvalchemiops.neighborlist import neighbor_list


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'lines.markersize': 4,
    'lines.linewidth': 1.5,  # Increased from 1 to 1.5
    'font.size': 8,
    'axes.labelsize': 8,
    'axes.titlesize': 8,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.titlesize': 8,
    'axes.grid': True,
    'grid.linewidth': 0.5,
    'grid.alpha': 1.0,
})

colors = sns.color_palette("deep")

In [ ]:
def time_fn(fn, n_tries=10, sync_cuda=False):
    # warmup
    fn()
    if sync_cuda and torch.cuda.is_available():
        torch.cuda.synchronize()

    times = []

    for _ in range(n_tries):
        if sync_cuda and torch.cuda.is_available():
            torch.cuda.synchronize()
        start = time.perf_counter()
        fn()
        if sync_cuda and torch.cuda.is_available():
            torch.cuda.synchronize()
        end = time.perf_counter()
        times.append((end - start) * 1000.0)  # ms

    if n_tries == 1:
        return times[0], 0.0

    mean_ms = sum(times) / n_tries

    # sample variance (no math)
    var = sum((t - mean_ms)**2 for t in times) / (n_tries - 1)

    # std and stderr via **0.5 instead of math.sqrt
    std_ms = var ** 0.5
    stderr_ms = std_ms / (n_tries ** 0.5)

    return mean_ms, stderr_ms

In [ ]:
cell = [
    [2.460394, 0.0, 0.0],
    [-1.26336, 2.044166, 0.0],
    [-0.139209, -0.407369, 6.809714],
]

positions = [
    [-0.03480225, -0.10184225, 1.70242850],
    [-0.10440675, -0.30552675, 5.10728550],
    [-0.05691216,  1.26093576, 1.70242850],
    [ 1.11473716,  0.37586124, 5.10728550],
]

base = Atoms(
    symbols="CCCC",
    positions=positions,
    cell=cell,
    pbc=True,
) * (4,4,2)


In [ ]:

number_of_structures = [1, 2, 4, 8, 16, 32, 64, 128]     
radii = [3.0, 4.5, 6.0]
n_tries = 4

device = "cuda:0" 

In [ ]:
results = {}   # radius → n_struct → timings dict
base_natoms = len(base)

for radius in radii:
    print("\n" + "=" * 80)
    print(f"radius = {radius}")
    print("=" * 80)

    results[radius] = {}

    for n_struct in number_of_structures:

        # Create n_struct independent copies of the same base structure
        configs = [base.copy() for _ in range(n_struct)]
        total_atoms = n_struct * base_natoms

        print(f"\nNumber of structures: {n_struct}  →  total_atoms = {total_atoms}")

        results[radius][n_struct] = {}

        # --- batch_nl NeighbourList (O(N^2), no compile) ---
        batch_nl = NeighbourList(
            list_of_positions=[s.positions for s in configs],
            list_of_cells=[s.cell.array for s in configs],
            cutoff=radius,
            device=device,
        )
        batch_nl.load_data()

        def batch_nl_no_compile():
            _ = batch_nl.calculate_neighbourlist(use_torch_compile=False)

        t_batch_nl_no_compile, e_batch_nl_no_compile = time_fn(
            batch_nl_no_compile, n_tries=n_tries, sync_cuda=True
        )
        results[radius][n_struct]["batch_nl_on2"] = {
            "mean": t_batch_nl_no_compile,
            "stderr": e_batch_nl_no_compile,
        }

        # --- batch_nl NeighbourList (O(N^2), compile=True) ---
        def batch_nl_compile():
            _ = batch_nl.calculate_neighbourlist(use_torch_compile=True)

        t_batch_nl_compile, e_batch_nl_compile = time_fn(
            batch_nl_compile, n_tries=n_tries, sync_cuda=True
        )
        results[radius][n_struct]["batch_nl_on2_compile"] = {
            "mean": t_batch_nl_compile,
            "stderr": e_batch_nl_compile,
        }

        # --- VESIN neighbour list (O(N)) ---
        def vesin_nl():
            for atoms in configs:
                i, j, S, D, d = vesin_ase_neighbor_list("ijSDd", atoms, cutoff=radius)

        t_vesin, e_vesin = time_fn(vesin_nl, n_tries=n_tries, sync_cuda=False)
        results[radius][n_struct]["vesin_on1"] = {
            "mean": t_vesin,
            "stderr": e_vesin,
        }

        # --- matscipy neighbour list (O(N)) ---
        def matscipy_nl():
            for atoms in configs:
                i, j, S, D, d = matscipy_neighbour_list("ijSDd", atoms, cutoff=radius)

        t_matscipy, e_matscipy = time_fn(
            matscipy_nl, n_tries=n_tries, sync_cuda=False
        )
        results[radius][n_struct]["matscipy_on1"] = {
            "mean": t_matscipy,
            "stderr": e_matscipy,
        }

        # --- shared batched data for torch-nl and NVIDIA ---
        pos, cell_t, pbc, batch_idx, _ = ase2data(configs)

        # --- torch-nl O(N) fully batched ---
        def torch_nl_n1():
            compute_neighborlist(radius, pos, cell_t, pbc, batch_idx, False)

        t_torch_n1, e_torch_n1 = time_fn(
            torch_nl_n1, n_tries=n_tries, sync_cuda=True
        )
        results[radius][n_struct]["torch_on1"] = {
            "mean": t_torch_n1,
            "stderr": e_torch_n1,
        }

        # --- NVIDIA ALCHEMI neighbour list (O(N), batched) ---
        def nvidia_nl():
            neighbor_matrix, num_neighbors, shifts = neighbor_list(
                pos,
                radius,
                cell=cell_t.contiguous().view(-1, 3, 3),
                pbc=pbc,
                batch_idx=batch_idx,
                method="batch_naive",
            )

        t_nvidia, e_nvidia = time_fn(
            nvidia_nl, n_tries=n_tries, sync_cuda=True
        )
        results[radius][n_struct]["nvidia_on1"] = {
            "mean": t_nvidia,
            "stderr": e_nvidia,
        }

        # summary
        print(
            f"  batch_nl O(N^2) (no compile)   : "
            f"{t_batch_nl_no_compile:8.3f} ± {e_batch_nl_no_compile:5.3f} ms"
        )
        print(
            f"  batch_nl O(N^2) (compile)      : "
            f"{t_batch_nl_compile:8.3f} ± {e_batch_nl_compile:5.3f} ms"
        )
        print(
            f"  VESIN O(N)                 : "
            f"{t_vesin:8.3f} ± {e_vesin:5.3f} ms"
        )
        print(
            f"  matscipy O(N)              : "
            f"{t_matscipy:8.3f} ± {e_matscipy:5.3f} ms"
        )
        print(
            f"  torch-nl O(N)              : "
            f"{t_torch_n1:8.3f} ± {e_torch_n1:5.3f} ms"
        )
        print(
            f"  NVIDIA O(N) (batch_cell)   : "
            f"{t_nvidia:8.3f} ± {e_nvidia:5.3f} ms"
        )

In [ ]:
methods = [
    "batch_nl_on2",
    "batch_nl_on2_compile",
    "vesin_on1",
    "matscipy_on1",
    "torch_on1",
    "nvidia_on1",
]

labels = {
    "batch_nl_on2":         "batch_nl O(N^2) [CUDA]",
    "batch_nl_on2_compile": "batch_nl O(N^2) + torch.compile [CUDA]",
    "vesin_on1":        "VESIN O(N) [CPU]",
    "matscipy_on1":     "matscipy O(N) [CPU]",
    "torch_on1":        "torch-nl O(N) + torch.jit [CUDA]",
    "nvidia_on1":       "NVIDIA ALCHEMI + warp [CUDA]",
}

# One color per method family (batch_nl_on2 + batch_nl_on2_compile share color)
colors = {
    "batch_nl_on2":     "tab:blue",
    "vesin_on1":    "tab:orange",
    "matscipy_on1": "tab:green",
    "torch_on1":    "tab:red",
    "nvidia_on1":   "tab:grey",
}

radii_list = list(results.keys())
num_panels = len(radii_list)

fig, axes = plt.subplots(
    1,
    num_panels,
    sharex=True,
    sharey=True,
    figsize=(3.41 * 2, 3.41),
)

if num_panels == 1:
    axes = [axes]

for ax, radius in zip(axes, radii_list):

    # x-axis = number of structures
    n_struct_list = sorted(results[radius].keys())

    for method in methods:

        means  = [results[radius][n][method]["mean"]   for n in n_struct_list]
        errors = [results[radius][n][method]["stderr"] for n in n_struct_list]

        # Color by family: batch_nl_on2 + batch_nl_on2_compile → "batch_nl_on2"
        if method.startswith("batch_nl_on2"):
            color_key = "batch_nl_on2"
        else:
            color_key = method

        color = colors[color_key]
        linestyle = "--" if "compile" in method else "-"

        ax.errorbar(
            n_struct_list,
            means,
            yerr=errors,
            marker="o",
            linestyle=linestyle,
            color=color,
            capsize=3,
            label=labels[method],
        )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(f"r = {radius}")
    ax.grid(True, which="both", ls="--", alpha=0.4)

for ax in axes:
    ax.set_xlabel("Number of structures")
axes[0].set_ylabel("Time (ms)")

handles, legend_labels = axes[0].get_legend_handles_labels()
fig.legend(handles, legend_labels, loc="upper center", ncol=len(methods))

plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.show()
